In [1]:
import pandas as pd
import numpy as np

In [2]:
virushostdb = pd.read_csv('virushostdb/virushostdb.tsv', sep="\t")
virushostdb.head()

,virus tax id,virus name,virus lineage,refseq id,KEGG GENOME,KEGG DISEASE,DISEASE,host tax id,host name,host lineage,pmid,evidence,sample type,source organism
0,438782,Abaca bunchy top virus,Viruses; Monodnaviria; Shotokuvirae; Cressdnav...,"NC_010314, NC_010315, NC_010316, NC_010317, NC...",NaN,NaN,NaN,214697.0,Musa acuminata AAA Group,Eukaryota; Viridiplantae; Streptophyta; Strept...,17978886,Literature,NaN,NaN
1,1241371,Abalone herpesvirus Victoria/AUS/2009,Viruses; Duplodnaviria; Heunggongvirae; Peplov...,NC_018874,NaN,NaN,NaN,36095.0,Haliotis diversicolor,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,28962967,Literature,NaN,NaN
2,491893,Abalone shriveling syndrome-associated virus,Viruses; Duplodnaviria; Heunggongvirae; Urovir...,NC_011646,NaN,NaN,NaN,37770.0,Haliotis diversicolor aquatilis,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,21079776,"Literature, RefSeq",NaN,NaN
3,11788,Abelson murine leukemia virus,Viruses; Riboviria; Pararnavirae; Artvervirico...,NC_001499,NaN,NaN,NaN,10090.0,Mus musculus,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,6257398,"Literature, UniProt",NaN,NaN
4,2025595,Abisko virus,Viruses; Riboviria; unclassified Riboviria,NC_035470,NaN,NaN,NaN,201501.0,Epirrita autumnata,Eukaryota; Opisthokonta; Metazoa; Eumetazoa; B...,NaN,"Literature, NCBI Virus, RefSeq",NaN,NaN


In [3]:
df = virushostdb[virushostdb["KEGG DISEASE"].notnull()][["virus tax id", "KEGG DISEASE"]]

df.rename(columns={"virus tax id": "virus_taxid"}, inplace=True)

taxid_disease = set()

diseases = set()

for index, row in df.iterrows():
    if "," in row['KEGG DISEASE']:
        ds = row['KEGG DISEASE'].split(",")

        for d in ds:
            #content += f'{row["virus_taxid"]},{d.strip()}\n'
            taxid_disease.add((row["virus_taxid"], d.strip()))

            diseases.add(d.strip())
    else:
        taxid_disease.add((row["virus_taxid"], row['KEGG DISEASE'].strip()))

        diseases.add(row['KEGG DISEASE'].strip())


In [4]:
content = "taxid,disease_ko\n"

for t_d in taxid_disease:
    content += f"{t_d[0]},{t_d[1]}\n"

with open("import/virus_disease.csv", "w") as output:
    output.write(content)

In [5]:
kegg_disease = pd.read_csv("parsed_file/disease.csv")

In [6]:
kegg_disease[kegg_disease["ko"].isin(diseases)].drop("icd10", axis=1).to_csv("import/disease.csv", index=False)

In [7]:
kegg_disease_drug = pd.read_csv("parsed_file/drug_disease.csv")
kegg_disease_drug.head()

viral_disease_drug = kegg_disease_drug[kegg_disease_drug["to"].isin(diseases)]

viral_disease_drug.to_csv("import/drug_disease.csv", index=False)

In [8]:
viral_disease_drug.head()

,from,to
7,D11014,H00413
53,D12014,H02398
61,D00088,H00366
89,D05529,H01418
105,D02764,H00366


In [9]:
kegg_drug = pd.read_csv("parsed_file/drug.csv")
kegg_drug.head()

kegg_drug[kegg_drug["ko"].isin(viral_disease_drug["from"].unique())].to_csv("parsed_file/drug_temp.csv", index=False)